In [ ]:
import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import json
import time
from random import randint
from time import sleep
from csv import writer

In [ ]:
# Headers for the request to represent it was requested by a browser
headers = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'he-IL,he;q=0.9,en-US;q=0.8,en;q=0.7',
}

# Funtion to make 'get' request for the url and return it as a soup 
def load_soup_object(url_path):
    page = requests.get(url_path, headers = headers)
    #print(page)
    return BeautifulSoup(page.content, "html.parser")

In [ ]:
# Function to messure the time spent for crawling
def time_convert(sec):
    mins = sec // 60
    sec = sec % 60
    hours = mins // 60
    mins = mins % 60
    print("Time Lapsed = {0}:{1}:{2}".format(int(hours),int(mins),sec))

In [ ]:
#Creating the data frame
dataframe = pd.DataFrame(columns = ['Name','Marital Status', 'Reality', 'Gender', 'Hair Color',
                                    'Eye Color', 'Living status', 'Identity', 'Citenzenship', 'Year', 'Appearnces'])

# Url prefix for each link in the page
url_prefix = 'https://marvel.fandom.com'

# Flag to mark the first page, (the first page contains 18 links that are not character pages)
first_page = True

# Start the timer
start_time = time.time()

#
next_page = None

In [ ]:
# Loading the first page
soup = load_soup_object('https://marvel.fandom.com/wiki/Category:Characters')

# Flag to represent if there is still a "Next" page.
not_last_page = True
while not_last_page:
    
    # Wrapper that hold all the links in the page
    current_page_character_list = soup.find_all('div', {'class': 'category-page__members-wrapper'})
    
    # Soup with all the links in the current page
    if first_page:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})[18:]
        first_page = False
    else:
        links = soup.find_all('a',  {'class': 'category-page__member-link'})

    # Loop that runs on all links that a page has    
    for i,X in enumerate(links):
        # Create new list to represent the each row of the data.
        row = list()
        #print(str(i) + ". " + url_prefix + X.get('href'))
        
        #Try and CCatch statement incase the soup loading failed.
        try:
            character_page_soup = load_soup_object(url_prefix + X.get('href'))
        except:
            print("An exception occurred during loading character page")

         # Adding the name of the character  
        if character_page_soup.find('div',  {'data-source': 'Name'}) is not None:
            if character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'}).a is not None:
                name = character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'}).a
                row.append(name.get_text().strip())
            else:
                name = character_page_soup.find('div',  {'data-source': 'Name'}).find('div',  {'class': 'pi-data-value pi-font'})
                name.get_text().strip()
                row.append(name.get_text().strip())
        else:
            row.append(None)

        # Adding the Marital Status of the character
        if character_page_soup.find('div',  {'data-source': 'MaritalStatus'}) is not None:
            if character_page_soup.find('div', {'data-source': 'MaritalStatus'}).a is not None: 
                m_status = character_page_soup.find('div', {'data-source': 'MaritalStatus'}).a
                row.append(m_status.get_text().strip())
            else:
                row.append(None)
        else:
            row.append(None)

        # Adding the Reality of the character
        if character_page_soup.find('div',  {'data-source': 'Reality'}) is not None: 
            reality_content = character_page_soup.find('div',  {'data-source': 'Reality'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            row.append(reality_content)
        else:
            row.append(None)

        # Adding gender of the character
        if character_page_soup.find('div',  {'data-source': 'Gender'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Gender'}).a is not None:
                gender = character_page_soup.find('div', {'data-source': 'Gender'}).a
                row.append(gender.get_text().strip())
            else:
                row.append(None)
        else:
            row.append(None)

        # Adding hair color of the character
        if character_page_soup.find('div',  {'data-source': 'Hair'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Hair'}).a is not None:
                hair = character_page_soup.find('div', {'data-source': 'Hair'}).a
                row.append(hair.get_text().strip())
            else:
                row.append(None)
        else:
            row.append(None)

        # Adding eye color of the character
        if character_page_soup.find('div',  {'data-source': 'Eyes'}) is not None:
            if character_page_soup.find('div', {'data-source': 'Eyes'}).a is not None:
                eye = character_page_soup.find('div', {'data-source': 'Eyes'}).a
                row.append(eye.get_text().strip())
            else:
                row.append(None)
        else:
            row.append(None)

        # Adding living status of the character
        if character_page_soup.find('div',  {'data-source': 'Status'}) is not None:
            status = character_page_soup.find('div',{'data-source': 'Status'}).find_all('a')
            row.append(status[0].get_text().strip())
        else:
            row.append(None)

        # Adding Identity status of the character
        if character_page_soup.find('div',  {'data-source': 'Identity'}) is not None:
            if character_page_soup.find('div',  {'data-source': 'Identity'}).a is not None:
                identity =  character_page_soup.find('div',  {'data-source': 'Identity'}).a.get_text().strip()
                row.append(identity)
            else:
                row.append(None)
        else:
            row.append(None)
            
        # Adding Citizenship of the character
        if character_page_soup.find('div',  {'data-source': 'Citizenship'}) is not None:
            citizen_content = character_page_soup.find('div',  {'data-source': 'Citizenship'}).find('div',  {'class': 'pi-data-value pi-font'}).get_text().strip()
            row.append(citizen_content)
        else:
            row.append(None)

        # Adding Year of the character first apearence
        if character_page_soup.find('div',  {'data-source': 'First'}) is not None:
            year_content = character_page_soup.find('div', {'data-source': 'First'}).find_all('a')
            if(len(year_content) > 1):
                year = character_page_soup.find('div', {'data-source': 'First'}).find_all('a')[1].get_text().strip().split()[-1]
            else:
                if len(character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()) > 0:
                    year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()[-1]
                    if 'Mentioned' in year:
                        year = year[0:4]
                    elif 'Carving' in year:
                        year = year[0:4]
                    elif 'Series' in year:
                        year = year[0:4]
                    else:
                        year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip().split()[-1][:-1]
                else:
                    year = character_page_soup.find('div', {'data-source': 'First'}).get_text().strip()
            row.append(year)
        else:
            row.append(None)
            
            
        # Adding the number of appearences of the character in comic books
        if character_page_soup.find('span',  {'id': 'See_Also'}) is not None:
            appearnces_parent = character_page_soup.find('span', {'id': 'See_Also'}).parent
            if appearnces_parent.next_sibling.next_sibling.find('li') is not None:
                appearnces_content = appearnces_parent.next_sibling.next_sibling.find('li').get_text().strip().split()[0]
                row.append(appearnces_content.split()[0])
            else:
                row.append(None)
        else:
            row.append(None)

        # appending the row to the dataframe
        series = pd.Series(row, index = dataframe.columns)
        dataframe = dataframe.append(series, ignore_index=True)
        
        # writing to the directly to a diffrent CSV for a backup
        with open(r'C:\Users\guyge\Desktop\data science\Marvel HIT\Data-Science-HIT-project\Marvel CSV.csv', 'a', newline='') as f_object:  
            # Pass the CSV  file object to the writer() function
            writer_object = writer(f_object)
            # Result - a writer object
            # Pass the data in the list as an argument into the writerow() function
            try:
                writer_object.writerow(row)
            except:
                print("Didn't write a row to csv")
            # Close the file object
            f_object.close()

   # Opening the next page
    if soup.find('a', {'class':'category-page__pagination-next'}) is not None:
        next_page = soup.find('a', {'class':'category-page__pagination-next'}).get('href')
        try:
            soup = load_soup_object(next_page)
        except:
            print("An exception occurred during loading next page")
    
    else:
        not_last_page = False


In [ ]:
# Printing the last page the crawler was on in case the was an exception,
# so we can continue from where it ended
print(next_page)

In [ ]:
# calculate the time and print the timer
end_time = time.time()
time_lapsed = end_time - start_time
time_convert(time_lapsed)

In [ ]:
#the final dataframe
dataframe

In [ ]:
#Saving the dataframe to CSV.
dataframe.to_csv('Marvel Dataframe 2.csv')